In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

In [3]:
import tensorflow as tf

In [4]:
import tensorflow.keras.backend as K

In [5]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
import matplotlib.pyplot as plt

# load model

In [7]:
# create model
base_model_front = EfficientNetB0(include_top=False, input_shape=(224, 224, 3))
base_model_front._name = 'EfficientNetB0_front'


model_front = tf.keras.Sequential([
    base_model_front,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1)
])

for layer in base_model_front.layers:
    layer._name = layer.name + "_front"

for layer in model_front.layers:
    layer._name = layer.name + "_front"

model_front.input._name = model_front.input.name + "_front"

In [8]:
# create model
base_model_side = EfficientNetB0(include_top=False, input_shape=(224, 224, 3))
base_model_side._name = 'EfficientNetB0_side'


model_side = tf.keras.Sequential([
    base_model_side,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1)
])

for layer in base_model_side.layers:
    layer._name = layer.name + "_side"

for layer in model_side.layers:
    layer._name = layer.name + "_side"

model_side.input._name = model_side.input.name + "_side"

In [9]:
model_structured = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(10)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1)
])

for layer in model_structured.layers:
    layer._name = layer.name + "_structured"

In [10]:
_x = tf.keras.layers.concatenate([model_front.layers[-2].output, model_side.layers[-2].output, model_structured.layers[-3].output])
top_model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1)
])
_x = top_model(_x)

In [11]:
model = tf.keras.Model(inputs=[model_front.input, model_side.input, model_structured.input], outputs=_x)

In [12]:
model.load_weights('models/ensemble_asian-v1-1')

In [14]:
model.compile(loss='mse', metrics=['mae'])

# take image and make input data

In [15]:
import cv2

In [16]:
# take front photo with camera
cap = cv2.VideoCapture(0)

front = None
side = None

while True:
    ret, frame = cap.read()
    cv2.imshow('frame', frame)
    # if the 'f' key is pressed, save the front photo to a variable
    if cv2.waitKey(1) & 0xFF == ord('f'):
        print('front photo taken')
        front = frame
    # if the 's' key is pressed, save the side photo to a variable
    if cv2.waitKey(1) & 0xFF == ord('s'):
        print('side photo taken')
        side = frame
    if front is not None and side is not None:
        break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

front photo taken
front photo taken
side photo taken


In [17]:
# close all windows
cap.release()
cv2.destroyAllWindows()

In [18]:
#  #   Column              Non-Null Count  Dtype  
# ---  ------              --------------  -----  
#  0   ID                  69657 non-null  object 
#  1   Height_cm           69657 non-null  float64
#  2   Weight_kg           69657 non-null  float64
#  3   Sex_Female          69657 non-null  bool   
#  4   Sex_Male            69657 non-null  bool   
#  5   Race_Amer Indian    69657 non-null  bool   
#  6   Race_Asian          69657 non-null  bool   
#  7   Race_Bi-Racial      69657 non-null  bool   
#  8   Race_Black          69657 non-null  bool   
#  9   Race_Hispanic       69657 non-null  bool   
#  10  Race_Not Available  69657 non-null  bool   
#  11  Race_White          69657 non-null  bool 

In [23]:
# make predictions
front = cv2.resize(front, (224, 224))
side = cv2.resize(side, (224, 224))

front = cv2.cvtColor(front, cv2.COLOR_BGR2RGB)
side = cv2.cvtColor(side, cv2.COLOR_BGR2RGB)

columns = ['Height_cm', 'Sex_Female', 'Sex_Male', 'Race_Amer Indian', 'Race_Asian',
           'Race_Bi-Racial', 'Race_Black', 'Race_Hispanic', 'Race_Not Available', 'Race_White']
sample_df = pd.DataFrame([[171,0,1,0,1,0,0,0,0,0]], columns=columns)

In [24]:
sample_df

,Height_cm,Sex_Female,Sex_Male,Race_Amer Indian,Race_Asian,Race_Bi-Racial,Race_Black,Race_Hispanic,Race_Not Available,Race_White
0,171,0,1,0,1,0,0,0,0,0


In [25]:
x = (np.expand_dims(front, 0), np.expand_dims(side, 0), tf.constant(
    sample_df.values, dtype=tf.float32))

y = model.predict(x)

1/1 [==============================] - 0s 121ms/step


In [26]:
y

array([[64.70343]], dtype=float32)